# Automatic transcription of mp4 file

Automatic speech recognition via Google Cloud speech-to-text.

Mp4 file is converted into a flac sample, uploaded to Google Cloud Storage and than transcribed.

In [11]:
!pip install google-cloud-speech -q
!pip install google-cloud-storage -q

In [2]:
from google.cloud import speech_v1 as speech
from google.oauth2 import service_account
from google.cloud import storage

In [3]:
creds = service_account.Credentials.from_service_account_file("google_credentials.json")

## Prepare input files

A 2 minute sample is created to ascertain the speech recognition quality. The mp4 video is converted to flac and uploaded to Google Cloud Storage.

In [4]:
name = "focus"

sample_start = 60*49
sample_length = 118

video_full = f"data/video_full/{name}.mp4"
video_sample = f"data/video_sample/{name}.mp4"
audio_sample = f"data/audio_sample/{name}.flac"

In [5]:
!"lib/ffmpeg" -i "{video_full}" -ss "{sample_start}" -t "{sample_length}" -acodec copy -vcodec copy "{video_sample}" -loglevel warning -hide_banner

In [6]:
!"lib/ffmpeg" -i "{video_sample}" -acodec flac "{audio_sample}" -loglevel warning -hide_banner

[flac @ 0000027543e7c080] encoding as 24 bits-per-sample
[flac @ 0000027543e7e380] Video stream #0 is not an attached picture. Ignoring


_upload manually to Google Cloud Storage_

## Google Cloud speech-to-text via api

In [7]:
config = {
    "language_code": "cs-CZ",
    "audio_channel_count": 2,
    "use_enhanced": True,
    }
audio ={
    "uri": f"gs://akademicky-kvalitativni-vyzkum-20/{name}.flac"
    }

speech_client = speech.SpeechClient(credentials=creds)
response = speech_client.long_running_recognize(config=config, audio=audio)

In [12]:
def callback(operation_future):
    result = operation_future.result()

response.add_done_callback(callback)
response.metadata.progress_percent

100

_once the operation is finished, check the transcription_

In [13]:
for result in response.result().results:
    best_alternative = result.alternatives[0]
    print(best_alternative.transcript)

a vlastně ještě nic nevím a mám toho profesora který nám před dvěma týdny ve středu řekl že pokud nepůjdeme v tuhle středu do školy tak neví jestli budeme teď ve středu psát
 zlatý mapy a řekl nám že nám dá ve středu vědět a teď jsme od nich žádný email nedostali a ten člověk má ještě ani jeden neodepsal na email a to už mi přijde trošku jako to jako ne že jako takový divný Mně to přijde od toho čekala jako ne jenom že mi přijde že ztrácím či že já ztrácím k němu nějakou jako úst nebo něco takového že prostě není tak těžké odepsat na tuhle dobu Já chápu že neodepíšeš na email když od ní Od rána do večera do školy vyučuje
 ale když utáhnu s ní vlastně denně u toho počítače a víš že vlastně ten email a teďkon jediná komunikace s těmi studentami tak
 na den a Znám pár lidí od nás z oboru nebo jako přímo specializace že jim taky neodepsal
 že nevím a já už musela nechce že se radši učím aby náhodou nepřišla ve středu tak si napíšeme mapy
 mají ještě podobnou zkušenost s tím že ta komunikac